<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-soyoungcareer/blob/main/week2/Chapter2_2_%EA%B3%BC%EC%A0%9CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 수행 순서
    
1. `collate_fn`  함수 수정
    1. labels, texts 설정
    2. pad-sequence 사용
2. TextClassifier 수정
    1. self.classification 출력 크기 수정
    2. 문장 끝에 해당하는 마지막 토큰을 예측
    3. 손실함수 변경



# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from torch.nn.utils.rnn import pad_sequence


ds = load_dataset("stanfordnlp/imdb")    # IMDB dataset : 기존 라벨은 긍정과 부정을 나타내는 이진 값.
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') # use this instead of torch.hub.load


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    tokenized = tokenizer(row['text'], padding=True, truncation=True, max_length=max_len)
    input_ids = tokenized.input_ids

    # 1-a. label = text의 마지막에서 두 번째 token 의 id로 설정 -> [SEP] 토큰을 제외하기 위해 하나를 더 뺌.
    labels.append(input_ids[-2])
    # 1-a. 마지막 두 token 제외한 나머지 token list
    texts.append(input_ids[:-2])

  # 1-b. pad-sequence 사용
  texts = [torch.LongTensor(t) for t in texts]
  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
  labels = torch.LongTensor(labels)

  return texts, labels

batch_size = 64

train_loader = DataLoader(
    ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [3]:
from torch import nn
from math import sqrt

# Self-attention -> Multi-head attention 으로 확장
class MultiHeadAttention(nn.Module):
  # n_heads 인자 추가
  def __init__(self, input_dim, d_model, n_heads):
    super().__init__()

    # self.input_dim = input_dim    # 입력차원 사용되지 않음.

    self.n_heads = n_heads    # n_heads 인자 추가
    self.d_head = d_model // n_heads    # 각 head가 한 번에 처리하는 정보의 크기(D' = D/H)
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    batch_size = x.size(0)    # 입력 데이터 x의 첫 번째 차원 = 배치 크기

    # q, k, v = self.wq(x), self.wk(x), self.wv(x)
    # x를 Linear layer에 통과
    # self.wq(x) : x를 Query 로 변환 (batch_size, seq_len, d_model) -> (B, S, D)
    # view : 텐서의 모양을 바꾸는 연산 -> (B, S, H, D')
    # transpose : view에서 나온 텐서의 각 차원을 교환하는 작업 -> (B, H, S, D')

    # reshape (각 헤드가 독립적으로 처리할 수 있게 함)
    # (B, S, D) -> (B, S, H, D') -> (B, H, S, D')
    q = self.wq(x).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
    k = self.wk(x).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
    v = self.wv(x).view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)

    # Attention score 계산 - Q, K 의 내적 계산(두 단어의 연관도)
    score = torch.matmul(q, k.transpose(-1, -2))   # (B, S, D) * (B, D, S) = (B, S, S)

    # 점수를 너무 커지지 않게 조정 - 기존 sqrt(self.d_model) 에서 sqrt(self.d_head) 로 변경
    score = score / sqrt(self.d_head)    # d_head = D'

    # 마스크가 있으면 해당하는 부분은 아주 작은 숫자로 바꿔서 무시되도록 함
    if mask is not None:
      mask = mask.unsqueeze(1)
      score = score + (mask * -1e9)

    # softmax를 사용해 모든 점수 합이 1이 되게 맞추고, 점수가 높은 부분을 강조함
    score = self.softmax(score)

    # 점수를 Value에 곱해서 최종 결과를 구함 (강조된 정보로 새로운 값을 만듦)
    result = torch.matmul(score, v)    # (B, S, S) * (B, S, D) = (B, S, D)

    # 여러 head에서 나온 결과를 다시 하나로 합침
    result = result.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

    # 마지막으로 Linear 레이어에 통과시켜서 최종 결과를 만듦
    result = self.dense(result)

    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [4]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff, n_heads, dropout=0.1):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    # MHA와 FFN을 거치면서 잘못된 부분은 버리고 중요한 정보를 남기는 정리 작업을 함.
    # - MHA : 여러 정보를 한 번에 비교
    # - FFN : 정보를 더 똑똑하게 바꾸는 것
    self.mha = MultiHeadAttention(input_dim, d_model, n_heads)    # MHA로 수정
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

    # Layer Normalization, Dropout 추가
    self.layer_norm1 = nn.LayerNorm(d_model)
    self.layer_norm2 = nn.LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask):
    # Residual connection : 원래 정보를 잠시 옆에 두었다가 나중에 다시 가져와서 새로운 정보와 합침. 정보를 잊어버리지 않게 해줌.
    # Layer Normalization : 정보를 정리해서 균형 있게 만들어줌.

    # Layer Normalization, Dropout, Residual connection 추가
    # Residual connection + Layer Normalization for Multi-head attention
    x1 = self.mha(x, mask)    # MHA로 수정
    x1 = self.dropout(x1)
    x = self.layer_norm1(x1 + x)

    # Residual connection + Layer Normalization for Feed-forward network
    x2 = self.ffn(x)
    x2 = self.dropout(x2)
    x = self.layer_norm2(x2 + x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [5]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [6]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff, n_heads):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff, n_heads) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, vocab_size)    # 2-a. 어휘 크기만큼 출력

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model)
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    # 2-b. 문장 끝에 해당하는 마지막 토큰을 예측
    x = x[:, -1]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 5, 32, 4)    # 5-layer 4-head Transformer

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [7]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()    # 1-c. 손실함수 변경(다중 클래스 분류)

optimizer = Adam(model.parameters(), lr=lr)

In [8]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0    # 총 몇 개의 예측을 했는지 세는 변수
  acc = 0    # 맞춘 개수를 세는 변수

  for data in dataloader:
    inputs, labels = data    # 입력 데이터와 정답 데이터
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)    # 모델이 예측한 값
    preds = torch.argmax(preds, dim=-1)    # 예측한 값 중 가장 높은 확률을 가진 토큰을 선택
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]    # 총 정답의 개수를 센다
    acc += (labels == preds).sum().item()    # 정답과 예측이 일치한 개수를 센다

  return acc / cnt    # 맞춘 개수 / 총 개수 = 정확도

In [ ]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 466.5285152923316
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   1 | Train Loss: 2.267855491489172
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   2 | Train Loss: 0.8090491169132292
=========> Train acc: 1.000 | Test acc: 1.000
Epoch   3 | Train Loss: 0.4060668842867017
=========> Train acc: 1.000 | Test acc: 1.000


KeyboardInterrupt: 

**Accuracy 가 계속 1로 나와서 이상함...**

*`-> [SEP] 토큰이 계속 label로 설정되어 정확도가 1로 나오는 것으로 보여 label 수정하였음.`*



학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.

In [9]:
dataiter = iter(train_loader)
texts, labels = next(dataiter)
print(tokenizer.convert_ids_to_tokens(labels))
print(labels)
print(tokenizer.convert_ids_to_tokens(texts[:, -1]))

['.', '.', '.', '.', '.', '.', '.', '##nar', '.', 'bad', 'good', '!', 'course', 'for', 'to', '.', '.', '.', '.', 'with', '>', '.', '!', 'in', '.', 'maurice', '.', '.', '.', '.', '!', '.', '.', '.', '.', 'about', '?', '.', 'panel', '<', ')', '.', ')', '!', '.', '.', '.', '>', '!', '!', 'anyone', '.', 'find', '.', '.', '.', '.', 'of', 'mabel', '.', '.', 'tell', '.', 'worst']
tensor([ 1012,  1012,  1012,  1012,  1012,  1012,  1012, 11802,  1012,  2919,
         2204,   999,  2607,  2005,  2000,  1012,  1012,  1012,  1012,  2007,
         1028,  1012,   999,  1999,  1012,  7994,  1012,  1012,  1012,  1012,
          999,  1012,  1012,  1012,  1012,  2055,  1029,  1012,  5997,  1026,
         1007,  1012,  1007,   999,  1012,  1012,  1012,  1028,   999,   999,
         3087,  1012,  2424,  1012,  1012,  1012,  1012,  1997, 19486,  1012,
         1012,  2425,  1012,  5409])
['[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '##ege', '[PAD]', 'more', 'with', '[PAD]', 'of', ',', '

In [10]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    loss = loss_fn(preds, labels)    # preds, labels 랑 차원이 맞는지 먼저 확인해봐야함.

    # debugging 방법
    # print(preds.shape)
    # print(labels.shape)
    # print(preds)
    # print(labels)

    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 1552.49851000309
=========> Train acc: 0.565 | Test acc: 0.565
Epoch   1 | Train Loss: 1020.8440598249435
=========> Train acc: 0.581 | Test acc: 0.580
Epoch   2 | Train Loss: 965.6580106019974
=========> Train acc: 0.588 | Test acc: 0.582
Epoch   3 | Train Loss: 925.2729852199554
=========> Train acc: 0.592 | Test acc: 0.584
Epoch   4 | Train Loss: 885.7412176132202
=========> Train acc: 0.600 | Test acc: 0.585
Epoch   5 | Train Loss: 845.3373807668686
=========> Train acc: 0.611 | Test acc: 0.582
Epoch   6 | Train Loss: 801.9570916891098
=========> Train acc: 0.624 | Test acc: 0.574
Epoch   7 | Train Loss: 755.7712087035179
=========> Train acc: 0.642 | Test acc: 0.566
Epoch   8 | Train Loss: 703.3891705274582
=========> Train acc: 0.672 | Test acc: 0.560
Epoch   9 | Train Loss: 654.8444178700447
=========> Train acc: 0.696 | Test acc: 0.536
Epoch  10 | Train Loss: 603.0409362912178
=========> Train acc: 0.732 | Test acc: 0.552
Epoch  11 | Train Loss: 551.5300

**=> Test accuracy 가 높아지지 않는 이유는 IMDB dataset 자체가 데이터수가 많지 않기 때문에 overfitting이 생길 수밖에 없음.**